In [ ]:
import os
import shutil
from contextlib import redirect_stdout

## Guides through the steps needed to process _.mp4_ video files in the VIDIMU dataset with [NVIDIA Maxine SDK](https://developer.nvidia.com/maxine) BodyTrack.

### 1. Print .mp4 filenames 

In [ ]:
subjects = ["S01","S02",  "S05","S06","S07","S08","S09","S10",
            "S11","S12","S13","S14","S15","S16","S17","S18","S19","S20",
            "S21","S22","S23","S24","S25","S26","S27","S28","S29","S30",
            "S31","S32","S33","S34","S35","S36","S37","S38","S39","S40",
            "S41","S42",  "S44",  "S46","S47","S48","S49","S50",
            "S51","S52","S53","S54","S55","S56","S57"]

In [ ]:
fulldatasetpathin = 'D:/VIDIMU'
fulldatasetpathout = 'D:/VIDIMU_UNPUBLISHED' #UNPUBLISHED OUTPUT OF BODYTRACK ARE BBOX OUTPUT VIDEOS

In [ ]:
inpath = os.path.join(fulldatasetpathin,'videosfullsize','videosoriginal')

In [ ]:
with open(os.path.join(inpath,'toBodyTrack.txt'),'w') as f:
    with redirect_stdout(f):
        for folder in sorted(os.listdir(inpath)):
            if folder in subjects:
                for file in os.listdir(os.path.join(inpath,folder)):
                    filename,extension = os.path.splitext(file)
                    if extension=='.mp4':
                        fullinname=os.path.join(inpath,folder,file)
                    else:
                        fullinname = 'Skip'
                        continue
                    print(fullinname)

### 2. Execute BodyTrack

- Copy `toBodyTrack.txt` to the path where BodyTrack.exe is located
- Execute the following code, within a .bat file (Windows)

```bash
SETLOCAL
SET PATH=%PATH%;..\..\samples\external\opencv\bin;..\..\bin;
for /f "delims=" %%f in (toBodyTrack.txt) do (
BodyTrack.exe --enable_people_tracking=0 --max_targets_tracked=1 --capture_outputs=true --model_path="C:\Program Files\NVIDIA Corporation\NVIDIA AR SDK\models" --offline_mode=true --mode=1 --use_cuda_graph=true --verbose=true --app_mode=1 --temporal=true --cam_res=640x480 --debug=false --in_file="%%f">"%%f".out 
)
```

### 3. Move files

In [ ]:
outpath_pose_out = os.path.join(fulldatasetpathin,'videosfullsize','videosbodytrack')
outpath_bbox = os.path.join(fulldatasetpathout,'videosfullsize','videosbodytrack')  #UNPUBLISHED

In [ ]:
for folder in os.listdir(inpath):
    if folder in subjects:
        for file in os.listdir(os.path.join(inpath,folder)):
            filename,extension = os.path.splitext(file)
            if extension=='.mp4':
                if '_bbox' in filename:
                    fullinname=os.path.join(inpath,folder,file)
                    fulloutfolder=os.path.join(outpath_bbox,folder)
                    fulloutname=os.path.join(fulloutfolder,file)
                    print(fullinname,'->',fulloutname)
                elif '_pose' in filename and '_bbox' not in filename:
                    fullinname=os.path.join(inpath,folder,file)
                    fulloutfolder=os.path.join(outpath_pose_out,folder)
                    fulloutname=os.path.join(fulloutfolder,file)
                else:
                    continue
            elif extension=='.out':
                fullinname=os.path.join(inpath,folder,file)
                fulloutfolder=os.path.join(outpath_pose_out,folder)
                fulloutname=os.path.join(fulloutfolder,file)
            else:
                continue
            print(fullinname,'->',fulloutname)
            if not os.path.exists(fulloutfolder):
                os.makedirs(fulloutfolder)
            shutil.move(fullinname,fulloutname)